# Practical 2 : Generative and Discriminative Models


In this practical, we will compare the Naïve Bayes Classifier (NBC) and Logistic Regression on several
datasets. As part of the practical you should read briefly the following paper:



**On Discriminative vs. Generative classifiers: A comparison of logistic regression
and naive Bayes**  
*Andrew Y. Ng and Michael I. Jordan*  
Advances in Neural Information Processing Systems (NIPS) 2001.

The paper is available on OLAT. 

You should read the Introduction and the Experiments sections. The goal of this practical is
to qualitatively reproduce some of the experimental results in this paper. You are strongly
encouraged to read the rest of the paper, which is rather short and straightforward to read,
though some of you may want to skip the formal proofs.

## Naïve Bayes Classifier (NBC)

You should implement a Naïve Bayes Classifier from scartch using NumPy. To keep your code tidy,
we recommend implementing it as a class. 
The classifier should be able to handle binary and continuous features. 
To earn the bonus points, the classifier should be able to handle categorical features as well. 
Suppose the data has 3
different features, the first being binary, the second being continuous and the third being categorical. Write an implementation that you can initialise as follows:

    nbc = NBC(feature_types=['b', 'r', 'c'])

Along the lines of classifiers provided in sklearn, you want to implement two more functions,
**fit** and **predict**. 
Recall the joint distribution of a generative model: $p(\mathbf{x}, y \mid \theta, \pi) = p(y \mid \pi) \cdot p(\mathbf{x} \mid y, \theta)$.
The **fit** function is to estimate all the parameters ($\theta$ and $\pi$) of the NBC, i.e., train the classifier. The **predict** function is to compute the probabilities that the new input belongs to all classes and
then return the class that has the largest probability, i.e., make the prediction.

    nbc.fit(X_train, y_train)
    ypredicted = nbc.predict(X_test)
    test_accuracy = np.mean(ypredicted == ytest)

Here we import the libraries. 

In [1]:
import sys, site, pathlib

print("sys.prefix:", sys.prefix)
print("sys.base_prefix:", sys.base_prefix)
print("in_venv:", sys.prefix != sys.base_prefix)

print("site-packages:")
for p in site.getsitepackages():
    print("  ", p)
import numpy as np
import pandas as pd
print("numpy:", np.__version__, "->", pathlib.Path(np.__file__).parents[1])
print("pandas:", pd.__version__, "->", pathlib.Path(pd.__file__).parents[1])

import pylab
pylab.rcParams['figure.figsize'] = (10., 10.)

import pickle as cp
import matplotlib.pyplot as plt

sys.prefix: /home/anabel/repositories/FoDS_Practicals/.venv
sys.base_prefix: /usr
in_venv: True
site-packages:
   /home/anabel/repositories/FoDS_Practicals/.venv/lib/python3.10/site-packages
   /home/anabel/repositories/FoDS_Practicals/.venv/local/lib/python3.10/dist-packages
   /home/anabel/repositories/FoDS_Practicals/.venv/lib/python3/dist-packages
   /home/anabel/repositories/FoDS_Practicals/.venv/lib/python3.10/dist-packages
numpy: 2.2.6 -> /home/anabel/repositories/FoDS_Practicals/.venv/lib/python3.10/site-packages
pandas: 2.3.3 -> /home/anabel/repositories/FoDS_Practicals/.venv/lib/python3.10/site-packages


### Class-conditional distributions

Before implementing NBC, we first implement the class-conditional distributions $p(\mathbf{x} \mid y, \theta)$. Your implementation should have two functions: **estimate** and **get_log_probability**. 

- The **estimate** function takes data as input and models the data using some distribution $p(x \mid \theta)$, where $\theta$ is the parameters of this distribution. The function estimates the parameters $\theta$ using maximum likelihood estimators (MLE). 
For example, in the case of continuous features, we use the Gaussian distribution to model the data. The estimate function will find the parameters $\mu$ and $\sigma$ for the Gaussian distribution with respect to the input data. 

- The **get_log_probability** function takes as input a new data point $x_{new}$ and returns the log of $p(x_{new} \mid \theta)$. For the Gaussian distribution, the function get_probability will return $\mathcal{N}(x_{new} \mid \mu, \sigma)$. 

For different types of features, you need to use different distributions.
You can import statistic libraries (e.g., `scipy.stats`) for the implementation of the distributions. 

- For **continuous features**: Use Gaussian distribution
    https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html
- For **binary features**: Use Bernoulli distribution 
    https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.bernoulli.html
- For **categorical features**: Use Multinoulli distribution (The multinoulli distribution is a special case of the multinomial distribution, where the number of trials is 1)
    https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.multinomial.html



**Implementation Issues:**
- The probabilities can be very small. To avoid underflow issues, you should compute the log of the probabilities. Read more: (Mur) Chapter 3.5.3 / Lecture Notes
- The standard deviation for Gaussian distributions should never be exactly 0, so in
case your calculated standard deviation is 0, you may want to set it to a small value such as 1e − 6. This is to ensure that your code never encounters division by zero or
taking logarithms of 0 errors. 
For this practical, please set the small value to 1e-6.
- Laplace/Additive smoothing: You want to ensure that the estimates for the parameter for the Bernoulli and Multinoulli random variables is never exactly 0 or 1. For this reason you should consider using Laplace smoothing (https://en.wikipedia.org/wiki/Additive_smoothing).
For this practical, please set alpha to 1.
- We will check the correctness of your implementation using the tests below.
- For simplicity, you can assume the data values for binary features are integers from {0,1} and the data for a categorical feature with M categories are integers from {0, ..., M-1}.
- Fell free to add auxiliary functions or change the parameters of the functions. If you change the parameters of the functions, make sure you change the tests accordingly, so we can test your code.


In [2]:
ALPHA = 1.0 #Laplace / additive smoothing parameter alpha https://en.wikipedia.org/wiki/Additive_smoothing

#Distribution for continuous features
class ContFeatureParam:
    def estimate(self, X):
        #Finding the Gaussian distribution 𝒩(𝑥;𝜇,𝜎^2) that fits the data using MLE 𝜃𝑗𝑐={𝜇,𝜎}

        self.mu = X.mean()
        sigma = X.std()

        #The standard deviation for Gaussian distributions should never be exactly 0, so in case your calculated standard deviation is 0, set it to 1e-6.
        self.sigma = max(sigma, 1e-6)

        return self

    def get_log_probability(self, X_new):
        #Computing 𝑝(𝑥𝑛𝑒𝑤^𝑗 | 𝑦=𝑐,𝜃𝑗𝑐) using 𝜃𝑗𝑐: Gaussian: 𝑝(𝑥𝑛𝑒𝑤^𝑗 | 𝑦=𝑐,𝜃𝑗𝑐) = 𝒩(𝑥𝑛𝑒𝑤^𝑗;𝜇,𝜎^2)
        #https://en.wikipedia.org/wiki/Normal_distribution: p(x) = (1 / sqrt(2𝜋𝜎^2)) * exp( -(x-μ)^2 / (2𝜎^2)
        #The probabilities can be very small. To avoid underflow issues, you should compute the log of the probabilities.
        #log 𝒩(𝑥;𝜇,𝜎^2) = -0.5 * [ log(2𝜋𝜎^2) + (x-μ)^2 / 𝜎^2 ]

        mu = self.mu
        sigma = self.sigma
        var = sigma ** 2
        log_p_x = -(0.5 * np.log(2 * np.pi * var)) -(((X_new - mu) ** 2) / (2 * var))

        return log_p_x


class BinFeatureParam:
    def estimate(self, X):

        n = X.size
        count1 = X.sum() #number of apperance of 1

        # Laplace/Additive smoothing: You want to ensure that the estimates for the parameter for the Bernoulli and Multinoulli random variables is never exactly 0 or 1. For this reason you should consider using Laplace smoothing (https://en.wikipedia.org/wiki/Additive_smoothing).
        self.p1 = (count1 + ALPHA) / (n + (2*ALPHA)) ##There are 2 possible values (0 and 1)

        return self

    def get_log_probability(self, X_new):
        #Bernoulli pmf:     p(x) = p1^x * (1 - p1)^(1-x) for x ∈ {0,1} https://en.wikipedia.org/wiki/Bernoulli_distribution
        #Taking logs:   log p(x) = x * log(p1) + (1 - x) * log(1 - p1) to avoid underflow

        p1 = self.p1
        p0 = 1 - p1

        return X_new * np.log(p1) + (1 - X_new) * np.log(p0)


class CatFeatureParam:
    def __init__(self, num_of_categories):
        self._num_of_categories = num_of_categories

    def estimate(self, X):
        #Estimation: 𝜃𝑗𝑐 = {𝑝(𝑥=𝑖),...(𝑥=𝑘)}
        #Prediction:  𝑝(𝑖 | 𝜃𝑗𝑐) = 𝑝(𝑥 = 𝑖)
        k = self._num_of_categories
        n = X.size

        #k distinct values in j-th column of the data where x ∈ {1, ..., k} and p(x = i) = # of apperance of i / # of data
        #counts[i] = number of times category i appears among the samples
        counts = np.bincount(X, minlength=k) #https://www.geeksforgeeks.org/python/numpy-bincount-python/

        # Laplace/Additive smoothing: You want to ensure that the estimates for the parameter for the Bernoulli and Multinoulli random variables is never exactly 0 or 1. For this reason you should consider using Laplace smoothing (https://en.wikipedia.org/wiki/Additive_smoothing).
        self.probs = (counts + ALPHA) / (n +(k*ALPHA)) #(k categories)

        return self

    def get_log_probability(self, X_new):
        log_probs = np.log(self.probs)
        return log_probs[X_new]

**Tests:**
    
We will use the code below to test the correctness of your code.

In [3]:
# continuous features

X = np.array([2.70508547,2.10499698,1.76019132,3.42016431,3.47037973,3.67435061,1.84749286,4.3388506,2.27818252,4.65165335])

param = ContFeatureParam()
param.estimate(X)
probs = param.get_log_probability(np.array([0,1,2,3]))
print(probs)

[-5.64670664 -3.02757918 -1.44567455 -0.90099277]


In [4]:
# binary features

X = np.array([0,0,1,1,0,1,0,1,1,1])

param = BinFeatureParam()
param.estimate(X)
probs = param.get_log_probability(np.array([0,1]))
print(probs)

[-0.87546874 -0.5389965 ]


In [5]:
# categorical features (bonus task)

X = np.array([0,6,5,4,0,6,6,4,1,1,2,3,8,8,1,6,4,9,0,2,2,3,8,0,2])

param = CatFeatureParam(num_of_categories=10)
param.estimate(X)
probs = param.get_log_probability(np.array([0,1,2,3,4,5,6,7,8,9]))
print(probs)

[-1.94591015 -2.1690537  -1.94591015 -2.45673577 -2.1690537  -2.86220088
 -1.94591015 -3.55534806 -2.1690537  -2.86220088]


### Implement NBC

We are now ready to implement NBC. We follow the structure of models in scikit-learn. We implement NBC as a class with functions **init**, **fit** and **predict**.
The **init** function takes as input the types of features and initialise the classifier. The **fit** function takes the training data as input and estimates the parameters. The **predict** function predicts the label for the input data. 

**Implementation Issues:**
- You should use matrix operations rather than loops. In general, loops over classes or features are OK, but loops over the rows of data are not a good idea.
- The probabilities can be very small. To avoid underflow issues, you should do the calculations in log space. Read more: (Mur) Chapter 3.5.3 / Lecture Note
- For simplicity, you can assume the data values for binary features are integers from {0, 1} and the data for a categorical feature with M categories are integers from {0, ..., M-1}.
- Fell free to add auxiliary functions or change the parameters of the functions. If you change the parameters of the functions, make sure you change the tests accordingly, so we can test your code.

In [6]:
# Your task is to implement the three functions of NBC. 

class NBC:
    # Inputs:
    #   feature_types: the array of the types of the features, e.g., feature_types=['b', 'r', 'c']
    def __init__(self, feature_types=[]):
        # We implement NBC as a class with functions **init**, **fit** and **predict**. The **init** function takes as input the types of features and initialise the classifier.
        self.feature_types = feature_types

        self.classes = None # set of class labels (e.g. [0,1] or [Beyonce,Borat,KanyeWest])
        self.log_class_prior = None # Class prior p(y = c | 𝜋)
        self.thetas_class_feature = None # θ = {θ_jc | for each class c and each feature j}
        self.num_categories = None # data for a categorical feature with M categories are integers from {0, ..., M-1}

        
    # The function uses the input data to estimate all the parameters of the NBC
    def fit(self, X, y):
        # fit function: Estimates all parameters
        #   𝜋={𝜋1,…,𝜋𝐶}
        #   𝜽={𝜃𝑗𝑐 |for each class 𝑐 and each feature 𝑗}
        # The **fit** function takes the training data as input and estimates the parameters.

        n_samples, n_features = X.shape

        if len(self.feature_types) != n_features:
            raise Exception("Number of feature types must match number of columns in X")

        # find classes and map y to indices 0, ..., C-1
        self.classes, y_indices = np.unique(y, return_inverse=True)
        classes_size = self.classes.size

        #𝝅c= p(y = c) = probability 𝜋𝑐 for each class 𝑐 = # of appearance of class c / # of data
        #Class prior: 𝑝(𝑦=𝑐 | 𝝅)
        class_counts = np.bincount(y_indices, minlength=classes_size)
        self.log_class_prior = np.log(class_counts / n_samples)

        # For simplicity, you can assume the data values for binary features are integers from {0, 1} and the data for a categorical feature with M categories are integers from {0, ..., M-1}
        self.num_categories = [None for _ in range(n_features)]
        for featureindex, featuretype in enumerate(self.feature_types):
            if featuretype == 'c':
                # categorical feature with M categories are integers from {0, ..., M-1}
                self.num_categories[featureindex] = int(X[:, featureindex].max()) + 1

        # θ = {θ_jc | for each class c and each feature j}
        self.thetas_class_feature = [[None for _ in range(n_features)] for _ in range(classes_size)]

        # You should use matrix operations rather than loops. In general, loops over classes or features are OK, but loops over the rows of data are not a good idea.

        """
        Pseudocode: fit

        function fit(X_train, y_train):
            for each class c:

                // estimate class prior
                pi_c <- p(y=c)

                for each feature j:

                    // get the data with class c and feature j
                    X_jc <- X_train[y_train==c, j]

                    // estimate theta_jc
                    // the estimation should be based on the type of j
                    theta_jc <- estimate theta_jc on X_jc
        """

        for classindex, classlabel in enumerate(self.classes):
            """// estimate class prior"""
            X_classlabel = X[y == classlabel]

            for featureindex, featuretype in enumerate(self.feature_types):
                """// get the data with class c and feature j"""
                X_classlabel_feature = X_classlabel[:, featureindex] # data used to estimate theta_jc

                """
                // estimate theta_jc
                // the estimation should be based on the type of j
                theta_jc <- estimate theta_jc on X_jc
                """
                if featuretype == 'r':
                    thetas_class_feature = ContFeatureParam().estimate(X_classlabel_feature)
                elif featuretype == 'b':
                    thetas_class_feature = BinFeatureParam().estimate(X_classlabel_feature)
                elif featuretype == 'c':
                    k = self.num_categories[featureindex]
                    thetas_class_feature = CatFeatureParam(num_of_categories=k).estimate(X_classlabel_feature)
                else:
                    raise Exception(f"Unknown feature type")

                self.thetas_class_feature[classindex][featureindex] = thetas_class_feature

        return self
                
                
    # The function takes the data X as input, and predicts the class for the data
    def predict(self, X):
        # The **predict** function predicts the label for the input data.

        n_samples, n_features = X.shape
        class_len = len(self.classes)

        #The probabilities can be very small. To avoid underflow issues, you should do the calculations in log space. Read more: (Mur) Chapter 3.5.3 / Lecture Note
        #predict function: For a new data 𝑥𝑛𝑒𝑤, computes for each class c:
        #𝑝(𝑦=𝑐|𝑥𝑛𝑒𝑤,𝜽,𝝅) = (𝑝(𝑦=𝑐|𝝅)⋅𝑝(𝑥𝑛𝑒𝑤|𝑦=𝑐,𝜽))/(𝑝(𝑥𝑛𝑒𝑤|𝜽,𝝅))
        #                = (𝜋𝑐*Π𝑗(𝑝(𝑥𝑛𝑒𝑤^j| 𝑦 =𝑐,𝜃𝑗𝑐))/(𝑝(𝑥𝑛𝑒𝑤|𝜽,𝝅))
        #log(𝑝(𝑦=𝑐|𝑥𝑛𝑒𝑤,𝜽,𝝅)) = log(𝜋𝑐)+∑𝑗(log(𝑝(𝑥𝑛𝑒𝑤^j| 𝑦 =𝑐,𝜃𝑗𝑐)))-log(𝑝(𝑥𝑛𝑒𝑤|𝜽,𝝅))
        #Choose the class with largest probability (no need to compute the denominator -log(𝑝(𝑥𝑛𝑒𝑤|𝜽,𝝅)) as it is same for all classes)

        """
        Pseudocode: predict

        function predict(x_new):

            for each class c:

                prob_c = pi_c

                for each feature j:

                    x_new_j = x_new[:,j]
                    prob_c *= p(x_new_j | theta_jc)

            return class c with the largest prob_c

        -In this pseudocode, the input of the predict function is a single data point.
        -In the skeleton code, the input of the predict function is a matrix with multiple data points. The
        function should predict the labels for all data points.
        """
        # You should use matrix operations rather than loops. In general, loops over classes or features are OK, but loops over the rows of data are not a good idea.

        log_lik_per_class = []

        for classindex in range(class_len):
            # log(𝑝(𝑥𝑛𝑒𝑤^j| 𝑦 =𝑐,𝜃𝑗𝑐)) for all features j
            logs_features = []
            for featureindex in range(n_features):
                theta_class_feature = self.thetas_class_feature[classindex][featureindex]
                logs_features.append(theta_class_feature.get_log_probability(X[:, featureindex]))

            # sum over features j = ∑𝑗(log(𝑝(𝑥𝑛𝑒𝑤^j| 𝑦 =𝑐,𝜃𝑗𝑐)))
            log_lik_per_class.append(np.sum(logs_features, axis=0))#rows

        log_lik_per_class = np.array(log_lik_per_class).T
        joint_log_likelihood = self.log_class_prior + log_lik_per_class # log(𝜋𝑐)+∑𝑗(log(𝑝(𝑥𝑛𝑒𝑤^j| 𝑦 =𝑐,𝜃𝑗𝑐)))

        #Choose the class with largest probability (no need to compute the denominator -log(𝑝(𝑥𝑛𝑒𝑤|𝜽,𝝅)) as it is same for all classes)
        indices = np.argmax(joint_log_likelihood, axis=1)#columns
        return self.classes[indices]


**Tests**

We will use the code below to check your code.

In [7]:
# All features of the iris dataset are continuous.

from sklearn.datasets import load_iris
iris = load_iris()
X, y = iris['data'], iris['target']

N, D = X.shape
Ntrain = int(0.8 * N)
Xtrain = X[:Ntrain]
ytrain = y[:Ntrain]
Xtest = X[Ntrain:]
ytest = y[Ntrain:]


nbc_iris = NBC(feature_types=['r', 'r', 'r', 'r'])
nbc_iris.fit(Xtrain, ytrain)
yhat = nbc_iris.predict(Xtest)
test_accuracy = np.mean(yhat == ytest)

print("Accuracy:", test_accuracy) # should be larger than 90%
print(yhat)

Accuracy: 0.9333333333333333
[2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [8]:
from sklearn.naive_bayes import GaussianNB
sk_nbG = GaussianNB()
sk_nbG.fit(Xtrain, ytrain)
yhat_sklearnG = sk_nbG.predict(Xtest)
acc_sklearnG = np.mean(yhat_sklearnG == ytest)

print("Accuracy:", acc_sklearnG)
print(yhat_sklearnG)

if np.array_equal(yhat,yhat_sklearnG):
    print("same")
else:
    print("different")

Accuracy: 0.9333333333333333
[2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
same


In [9]:
# All features of this dataset are binary
import pandas as pd
data = pd.read_csv('../datasets//binary_test.csv', header=None)
data = data.to_numpy()

X = data[:,1:]
y = data[:,0]

N, D = X.shape
Ntrain = int(0.8 * N)
Xtrain = X[:Ntrain]
ytrain = y[:Ntrain]
Xtest = X[Ntrain:]
ytest = y[Ntrain:]


nbc = NBC(feature_types=['b'] * 16)
nbc.fit(Xtrain, ytrain)
yhat = nbc.predict(Xtest)
test_accuracy = np.mean(yhat == ytest)

print("Accuracy:", test_accuracy) # should be larger than 85%
print(yhat)

Accuracy: 0.8723404255319149
[1 0 1 1 0 1 1 0 0 1 1 1 0 1 1 1 1 1 0 1 1 0 1 0 0 1 1 1 0 1 1 1 0 1 0 1 0
 0 0 1 0 0 0 0 1 1 0]


In [10]:
from sklearn.naive_bayes import BernoulliNB
sk_nbB = BernoulliNB(alpha=1.0)
sk_nbB.fit(Xtrain, ytrain)
yhat_sklearnB = sk_nbB.predict(Xtest)
acc_sklearnB = np.mean(yhat_sklearnB == ytest)

print("Accuracy:", acc_sklearnB)
print(yhat_sklearnB)

if np.array_equal(yhat,yhat_sklearnB):
    print("same")
else:
    print("different")

Accuracy: 0.8723404255319149
[1 0 1 1 0 1 1 0 0 1 1 1 0 1 1 1 1 1 0 1 1 0 1 0 0 1 1 1 0 1 1 1 0 1 0 1 0
 0 0 1 0 0 0 0 1 1 0]
same


In [11]:
# All features of this dataset are categorical (bonus task)

data = pd.read_csv('../datasets/categorical_test.csv', header=None)
data = data.to_numpy()

X = data[:,:-1]
y = data[:,-1]

N, D = X.shape
Ntrain = int(0.8 * N)
Xtrain = X[:Ntrain]
ytrain = y[:Ntrain]
Xtest = X[Ntrain:]
ytest = y[Ntrain:]


nbc = NBC(feature_types=['c'] * 9)
nbc.fit(Xtrain, ytrain)
yhat = nbc.predict(Xtest)
test_accuracy = np.mean(yhat == ytest)

print("Accuracy:", test_accuracy) # should be larger than 65%
print(yhat)

Accuracy: 0.6896551724137931
[0 1 0 1 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 1 1 0 1 0 1 0 0 0 0 1 0
 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0]


In [12]:
#scikit-learn CategoricalNB
from sklearn.naive_bayes import CategoricalNB
sk_nbC = CategoricalNB(alpha=1.0)
sk_nbC.fit(Xtrain, ytrain)
yhat_sklearnC = sk_nbC.predict(Xtest)
acc_sklearnC = np.mean(yhat_sklearnC == ytest)

print("Accuracy:", acc_sklearnC)
print(yhat_sklearnC)

if np.array_equal(yhat,yhat_sklearnC):
    print("same")
else:
    print("different")

Accuracy: 0.6896551724137931
[0 1 0 1 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 1 1 0 1 0 1 0 0 0 0 1 0
 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0]
same


## Logistic Regression

For logistic regression, you should use the implementation in scikit-learn. Add the following
line to import the LR model.

In [13]:
from sklearn.linear_model import LogisticRegression

Check the scikit-learn documentation for the Logistic Regression model:
- http://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
- http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression


## Comparing NBC and LR

### Experiments

The experiment is to compare the classification error of the NBC and LR trained on increasingly larger training datasets. 
Since the datasets are so small, you should do this multiple times and
average the classification error. One run should look as follows:
- Shuffle the data, put 20% aside for testing.
    
    ```N, D = X.shape
    Ntrain = int(0.8 * N)
    shuffler = np.random.permutation(N)
    Xtrain = X[shuffler[:Ntrain]]
    ytrain = y[shuffler[:Ntrain]]
    Xtest = X[shuffler[Ntrain:]]
    ytest = y[shuffler[Ntrain:]]
    
    ```  


- Train the classifiers with increasingly more data. For example, we can train classifiers with 10%, 20%, ..., 100% of the training data. For each case store the classification errors on the test set of the classifiers.

You may want to repeat this with at least 200 random permutations (possibly as large as 1000)
to average out the test error across the runs. In the end, you will get average test errors as a function of the size of the training data. 
We have written for you the function for making the plots for the experiments. 

In [14]:
# inputs:
#   nbc: Naive Bayes Classifier
#   lr: Logistic Regression Classifier
#   X, y: data
#   num_runs: we need repeat num_runs times and store average results
#   num_splits: we want to compare the two models on increasingly larger training sets.
#               num_splits defines the number of increasing steps. 
# outputs:
#   the arrays of the test errors across the runs of the two classifiers 
def compareNBCvsLR(nbc, lr, X, y, num_runs=200, num_splits=10):
    # The code below is just for compilation. 
    # You need to replace it by your own code.
    ###################################################
    ##### YOUR CODE STARTS HERE #######################
    ###################################################
    tst_errs_nbc = []
    tst_errs_lr = []
    
    return tst_errs_nbc, tst_errs_lr
    ###################################################
    ##### YOUR CODE ENDS HERE #########################
    ###################################################

In [15]:
def makePlot(nbc_perf, lr_perf, title=None, num_splits=10):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)

    ax.tick_params(axis='both', labelsize=20)

    ax.set_xlabel('Percent of training data used', fontsize=20)
    ax.set_ylabel('Classification Error', fontsize=20)
    if title is not None: ax.set_title(title, fontsize=25)

    xaxis_scale = [(i + 1) * (100/num_splits) for i in range(num_splits)]
    plt.plot(xaxis_scale, nbc_perf, label='Naive Bayes')
    plt.plot(xaxis_scale, lr_perf, label='Logistic Regression', linestyle='dashed')
    
    ax.legend(loc='upper right', fontsize=20)

### Datasets

Tasks: For each dataset,
1. Prepare the data for the two classifiers, e.g., handle missing values and the categorical data. When you handle the categorical data, you should check whether the data is ordinal or not. If the data is ordinal, you should encode the data as integers. If the data is not ordinal, you should encode the data as one-hot vectors.
2. Show the first 5 rows of the prepared data
3. Compare the two classifiers on the dataset and generate the plots

The grading will be based on whether the data is correctly prepared and the plots are generated without errors. The grading will not be based on the performance of the classifiers and whether the plots are the same as in the paper. 

**Dataset 1: Iris Dataset**

https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html

In [16]:
# TODO: insert your code for experiments
###################################################
##### YOUR CODE STARTS HERE #######################
###################################################
from sklearn.datasets import load_iris
iris = load_iris()


###################################################
##### YOUR CODE ENDS HERE #########################
###################################################

**Dataset 2: Voting Dataset**

https://archive.ics.uci.edu/ml/datasets/congressional+voting+records

The logistic regression line meets the naive bayes line early in the plot. 

In [17]:
# load the dataset
# TODO: insert your code for experiments
###################################################
##### YOUR CODE STARTS HERE #######################
###################################################
voting = pd.read_csv('./datasets/voting.csv')

voting.info()

###################################################
##### YOUR CODE ENDS HERE #########################
###################################################

FileNotFoundError: [Errno 2] No such file or directory: './datasets/voting.csv'

**Dataset 3: Breast Cancer Dataset (Bonus Tasks)**

https://archive.ics.uci.edu/ml/datasets/breast+cancer

The dataset has continues, binary and categorical features. It also has missing values.

Hints: You can precompute the size of the domains of the categorical features.

In [ ]:
# load the dataset
# TODO: insert your code for experiments
###################################################
##### YOUR CODE STARTS HERE #######################
###################################################
cancer = pd.read_csv('./datasets/breast-cancer.csv')

cancer.info()

###################################################
##### YOUR CODE ENDS HERE #########################
###################################################